In [3]:
import math
import random
import import_ipynb
import pandas as pd
import matplotlib.pyplot as plt
from Q1 import red_ks,yel_ks,blu_ks
import numpy as np

importing Jupyter notebook from Q1.ipynb


---
输入X/X0,Y/Y0,Z/Z0，计算LAB

In [4]:
def calculate_lab(XYZ_divided):
    X_ratio, Y_ratio, Z_ratio = XYZ_divided
    L_star = 116 * np.cbrt(Y_ratio) - 16
    a_star = 500 * (np.cbrt(X_ratio) - np.cbrt(Y_ratio))
    b_star = 200 * (np.cbrt(Y_ratio) - np.cbrt(Z_ratio))
    return np.array([L_star,a_star,b_star])

随机一组浓度，输出浓度和R

In [11]:
def ks(base_ks=None):
    # 配方的ks,1*16波长
    ks0 = [0.039492, 0.025906, 0.017964, 0.015092, 0.011439, 0.009515, 0.007961, 0.006947, 0.006284, 0.005889, 0.005238, 0.004948, 0.004626, 0.004247, 0.004100, 0.003617]
    arr = np.arange(16)
    if base_ks is None:
        red_des = round(np.random.uniform(0, 0.05), 4)
        yel_des = round(np.random.uniform(0, 0.05), 4)
        blu_des = round(np.random.uniform(0, 0.05), 4)
    # if base_ks is None:
    #     red_des = 0.0451
    #     yel_des = 0.0104
    #     blu_des = 0.0002
    else:
        red_des, yel_des, blu_des = base_ks
        red_des += round(np.random.uniform(-0.01, 0.01), 4)*0.5
        yel_des += round(np.random.uniform(-0.01, 0.01), 4)*0.5
        blu_des += round(np.random.uniform(-0.01, 0.01), 4)*0.5
        
        # 确保数据在有效范围内
        red_des = round(max(min(red_des, 0.05), 0),4)
        yel_des = round(max(min(yel_des, 0.02), 0),4)
        blu_des = round(max(min(blu_des, 0.02), 0),4)
    
    # 应该是每个列直接相加而非横向连接
    npks = np.sum([ks0, np.multiply(red_des,red_ks(arr, red_des)),np.multiply(yel_des,yel_ks(arr, yel_des)),np.multiply(blu_des,blu_ks(arr, blu_des))] ,axis=0)
    R = 1 + npks - np.sqrt(npks**2 + 2*npks)
    return R,red_des,yel_des,blu_des

In [12]:
ks()

(array([0.39962963, 0.39997097, 0.38829542, 0.40606752, 0.44056284,
        0.48437217, 0.53458499, 0.57844369, 0.60325975, 0.71015361,
        0.81039529, 0.84625505, 0.85938898, 0.87594688, 0.8959097 ,
        0.90516682]),
 0.0196,
 0.0467,
 0.0055)

目标样的lab

In [50]:
# def aim_LAB():
#     df_R = pd.read_excel("附件3.xlsx")
#     np_R[10,] = np.array(df_R)[1:11,1:17]
#     print(np_R)

In [51]:
# aim_LAB()

In [52]:
def bound_value(value, lower_bound, upper_bound):
    if value < lower_bound:
        return lower_bound + (lower_bound - value)
    elif value > upper_bound:
        return upper_bound - (value - upper_bound)
    else:
        return value

In [2]:
# ks()

---
输入16*1的R,计算配方的LAB，return目标样和标准样的LAB

In [54]:
# 先ks=ks()然后用LAB(ks),return LAB
def LAB(R):
    dfXYZ = pd.read_excel("附件1.xlsx")
    npXYZ = np.array(dfXYZ)
    Sxyz = npXYZ[2:18, 1:4]  # 16*3
    R = np.array([0.2310211 , 0.29080425, 0.31958438, 0.33157188, 0.33775017,
        0.33881281, 0.33541254, 0.35746793, 0.34957339, 0.52567033,
        0.58836073, 0.59606713, 0.61436572, 0.68442125, 0.74778616,
        0.80120968])
    R = R.reshape(-1, 1)  # reshape R to 16*1 for broadcasting
    result = np.multiply(R, Sxyz) * 20 * 0.1  # Broadcasting happens here
    lab_xyz = np.sum(result, axis=0)  # 标准样的xyz，一维长度3

    # 标准样判断，Xj/X0, Yj/Y0, Zj/Z0
    x0 = 94.83
    y0 = 100.00
    z0 = 107.38
    y_divided = lab_xyz / [x0, y0, z0]
    
    # 读取目标样和标准样数据
    lab = calculate_lab(y_divided)
    return lab
    

In [55]:
# R,red_des,yel_des,blu_des = ks()
# lab = LAB(R)
# print(R)
# print(lab)

>接下来到了重头戏，求色差

In [56]:
def aim(i, ks_values):
    R,red_des,yel_des,blu_des = ks_values
    lab_LAB=LAB(R)
    #print(R,lab_LAB)
    aim_df = pd.read_excel("目标样LAB.xlsx")
    aim_LAB = [np.array(aim_df)[i-1] for i in range(10)]
    #print(aim_LAB) #索引i，然后是10个目标样的LAB长度3
    # 计算色差
    color_differences = []
    
    diff = aim_LAB[i-1] - lab_LAB
    delta_L, delta_a, delta_b = diff
    delta_E = np.sqrt(delta_L**2 + delta_a**2 + delta_b**2)
    return red_des ,yel_des, blu_des,delta_E

def simulated_annealing(i, initial_temp=3000, final_temp=0.1, cooling_rate=0.99, max_iter_per_temp=200):
    """Simulated annealing algorithm for minimizing color difference.
    
    Args:
        i (int): The index of the aim.
        initial_temp (float, optional): The initial temperature for the simulated annealing. Default is 5000.
        final_temp (float, optional): The final temperature for the simulated annealing. Default is 0.1.
        cooling_rate (float, optional): The cooling rate for the simulated annealing. Default is 0.99.
        max_iter_per_temp (int, optional): The maximum number of iterations per temperature. Default is 100.

    Returns:
        tuple: The best solution and the corresponding color difference.
    """
    
    # Initialize the current solution and best solution
    current_solution = ks()
    best_solution = current_solution
    current_cost = aim(i, current_solution)[3]
    best_cost = current_cost
    

    # Main loop
    temp = initial_temp
    while temp > final_temp:
        for _ in range(max_iter_per_temp):
            # Generate a new solution randomly
            new_solution = ks(current_solution[1:])
            new_cost = aim(i, new_solution)[3]

            # If the new solution is better, update the current solution and best solution
            if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / temp):
                current_solution = new_solution
                current_cost = new_cost
                if new_cost < best_cost:
                    best_solution = new_solution
                    best_cost = new_cost
                    print(best_solution)
                    print(best_cost)

        # Cool down
        temp *= cooling_rate

    return best_solution, best_cost

In [57]:


# Test the function
print(simulated_annealing(7))

KeyboardInterrupt: 

In [1]:
# 定义数组
array = [
    [9.5e-05, 0.000429, 0.00394],
    [0.003385, 1.5e-05, 0.000249],
    [5e-05, 0.000321, 0.003407],
    [0.000602, 1.3e-05, 0.00225],
    [0.0002, 0.0013, 0.0104],
    [0.0004, 0.0003, 0.0076],
    [0.006, 0.0021, 0.0008],
    [0.0042, 0.0098, 0.0018],
    [0.0025, 0.0006, 0.0008],
    [0.0, 0.0004, 0.0023]
]

# 定义公式
def cost(red_des, yel_des, blu_des):
    return 2*(60*red_des + 65*yel_des + 63*blu_des)

# 计算总和
total_cost = sum(cost(red_des, yel_des, blu_des) for red_des, yel_des, blu_des in array)

print(total_cost)


8.304775999999999
